In [ ]:
import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

In [ ]:
# 훈련 세트를 6대 4로 나눕니다.
# 결국 훈련에 15,000개 샘플, 검증에 10,000개 샘플, 테스트에 25,000개 샘플을 사용하게 됩니다.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## 데이터 탐색

이 데이터셋의 샘플은 전처리된 정수 배열이다. 이 정수는 영화 리뷰에 나오는 단어를 나타낸다. 레이블은 정수 0 또는 1이다. 0은 부정 1은 긍정이다.

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [ ]:
train_labels_batch

## 모델 구성

신경망은 층을 쌓아서 만든다. 여기에는 3개의 중요한 구조적 결정이 필요하다.

> 어떻게 텍스트를 표현할 것인가

> 모델에서 얼마나 많은 층을 사용할 것인가

> 각 층에서 얼마나 많은 *은닉유닛*을 사용할 것인가

텍스트를 표현하는 한 가지 방법은 문장을 임베딩 벡터로 바꾸는 것이다. 그러면 첫번째 층으로 사전훈련된 텍스트 임베딩을 사용할 수 있다. 이것의 장점은 

> 텍스트 전처리에 신경 쓸 필요가 없다.

> 전이 학습의 장점을 이용한다.

> 임베딩은 고정 크기이기 때문에 처리 과정이 단순해진다.

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

1. 첫번째 층은 텐서플로 허브 층이다. 이 층은 사전 훈련된 모델을 사용하여 하나의 문장을 임베딩 벡터로 매핑한다. 여기서 사용하는 사전 훈련된 텍스트 임베딩 모델은 하나의 문장을 토큰으로 나누고 각 토큰의 임베딩을 연결하여 반환한다. 최종 차원은 (num_examples, embedding_demension)이다.

2. 이 고정 크기의 출력 벡터는 16개의 은닉 유닛을 가진 완전 연결층(Dense)로 주입된다.

3. 마지막 층은 하나의 출력 노드를 가진 완전 연결 층이다. sigmoid 활성화 함수를 사용하므로 확률 또는 신뢰도 수준을 표현하는 0~1 사이의 실수가 출력된다.

## 손실함수와 옵티마이저 

모델이 훈련하려면 손실함수와 옵티마이저가 필요하다. 지금은 이진 분류문제이고 확률을 출력하므로 binary_crossentropy 손실 함수를 사용한다.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 모델훈련

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)